In [1]:
%run bert.ipynb

attention pattern raw MATCH!!!!!!!!
 SHAPE (2, 12, 3, 3) MEAN: 0.0006189 STD: 0.1052 VALS [-0.1194 -0.0409 0.2213 0.125 -0.2653 -0.02375 -0.056 0.07237 -0.03704 0.01106...]
attention MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.000583 STD: 0.1108 VALS [0.016 -0.01431 0.1939 -0.007942 0.08188 -0.02789 0.07227 0.07465 0.1954 -0.2553...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.001554 STD: 0.1736 VALS [-0.08316 -0.09165 -0.03188 -0.03013 0.1001 0.09549 -0.1046 0.07742 0.0424 0.05553...]
bert mlp MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -0.0001934 STD: 0.1044 VALS [-0.1153 0.1189 -0.0813 0.1021 0.0296 0.06182 0.0341 0.1446 0.2622 -0.08507...]
layer norm MATCH!!!!!!!!
 SHAPE (20, 10) MEAN: -1.431e-08 STD: 1.003 VALS [0.6906 -0.84 1.881 1.711 -0.5116 -0.9577 -0.1387 -0.6943 -0.6741 -0.4662...]
bert MATCH!!!!!!!!
 SHAPE (2, 3, 768) MEAN: -5.381e-09 STD: 1 VALS [0.007132 -0.04372 0.6502 -0.5972 -1.097 0.7267 0.1275 -0.6035 -0.2226 0.2145...]
{'input_ids': [[101, 8667, 117, 146, 1821, 170, 565

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import transformers
from tqdm.notebook import tqdm
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
def make_bert():
    my_bert = Bert(
        vocab_size=28996, hidden_size=768, max_position_embeddings=512, 
        type_vocab_size=2, dropout=0.1, intermediate_size=3072, 
        num_heads=12, num_layers=12, num_classes=2
    )
    pretrained_state_dict = pretrained_bert.state_dict()

    #pretrained_keys = pretrained_state_dict.keys()
    #my_keys = my_bert.state_dict().keys()
    #print(set(my_keys) - set(pretrained_keys))

    # we could load these but actually we want to be fine-tuning this stuff in w2d2
    # so let's not take these 
    del pretrained_state_dict["classification_head.weight"]
    del pretrained_state_dict["classification_head.bias"]

    my_bert.load_state_dict(pretrained_state_dict, strict=False)
    
    return my_bert

In [4]:
def ascii_art_probs(sentence):
    bert = make_bert()
    token_ids = torch.Tensor(tokenizer.encode(sentence)).long().unsqueeze(0)
    mask_idxs = set()
    for idx, token_id in enumerate(token_ids[0]):
        if token_id == 103: # 103 == [MASK]
            mask_idxs.add(idx)
    unnormalized_output, _classifications = bert(token_ids)
    output = torch.log_softmax(unnormalized_output, dim=-1)
    top_k = torch.topk(output, 5, dim=-1)
    results = []
    for seq_i, seq_top_k in enumerate(top_k.indices[0]):
        if seq_i in mask_idxs:
            results.append(list(zip(tokenizer.convert_ids_to_tokens(seq_top_k), [round(x.item(), 2) for x in top_k.values[0][seq_i].exp()])))
    for l in results:
        print(l)

ascii_art_probs("The fish loves to eat [MASK] and [MASK] and [MASK] and [MASK] and [MASK] and [MASK] and [MASK].")


[('fish', 0.2), ('eggs', 0.08), ('meat', 0.07), (',', 0.07), ('apples', 0.03)]
[('fish', 0.14), ('eggs', 0.07), ('meat', 0.06), ('potatoes', 0.04), ('vegetables', 0.03)]
[('fish', 0.1), ('meat', 0.05), ('eggs', 0.03), ('salt', 0.03), ('bread', 0.02)]
[('fish', 0.08), ('eggs', 0.03), ('meat', 0.03), ('food', 0.02), ('salt', 0.02)]
[('fish', 0.06), ('eggs', 0.03), ('salt', 0.02), ('meat', 0.02), ('vegetables', 0.02)]
[('fish', 0.06), ('salt', 0.02), ('meat', 0.02), ('food', 0.02), ('water', 0.01)]
[('fish', 0.07), ('more', 0.03), ('honey', 0.02), ('meat', 0.02), ('salt', 0.01)]


In [5]:
# class ClassificationBERT():
#     def __init__(self, num_classes):
#         VOCAB_SIZE = 28996
#         DROPOUT=0.1
#         self.bert = Bert(
#             vocab_size=VOCAB_SIZE, hidden_size=768, max_position_embeddings=512, 
#             type_vocab_size=2, dropout=DROPOUT, intermediate_size=3072, 
#             num_heads=12, num_layers=12
#         )
#         self.dropout = torch.nn.Dropout(DROPOUT)
#         # TODO attach this to the last block of BERT as a hidden_size x num_classes linear layer,
#         # not on top of the vocab_size output of BERT
#         self.classification_head = torch.nn.Linear(VOCAB_SIZE, num_classes)

#         pretrained_state_dict = pretrained_bert.state_dict()
#         del pretrained_state_dict["classification_head.weight"]
#         del pretrained_state_dict["classification_head.bias"]
#         self.bert.load_state_dict(pretrained_state_dict)

#     def forward(self, X):
#         return self.classification_head(self.dropout(self.bert(X)))

In [6]:
import torchtext
import random
data_train, data_test = torchtext.datasets.IMDB(root='.data', split=('train', 'test'))
data_train_list = list(data_train)
random.shuffle(data_train_list)

In [7]:

def make_batches(data_train_list, batch_size, max_seq_len):
    batches = []
    for batch_start in range(0, len(data_train_list), batch_size):
        batch_end = min(batch_start + batch_size, len(data_train_list))
        ratings, sentences = [list(tup) for tup in list(zip(*data_train_list[batch_start:batch_end]))]
        tokens = tokenizer(sentences, padding='longest', max_length=max_seq_len, truncation=True)
        ratings01 = torch.tensor([ rating == 'pos' for rating in ratings ], dtype=torch.long)
        tokens_tensor = torch.tensor(tokens.input_ids, dtype=torch.long)
        batches.append((tokens_tensor, ratings01))
    return batches

In [8]:
for review, sentiment in make_batches(data_train_list[:5], 1, 100):
    print(tokenizer.decode(review[0]), sentiment)

[CLS] It is a great tragedy that both Richard Harris and John Derek are no longer with us. But that shouldn't blind anybody to the fact that in 1981, a pretty ugly blotch appears on both men's CVs. No doubt John Derek conceived this movie doing for his wife what'Some Like it Hot'and'One Million Years BC'did for Maryln Monroe and Raquel Welsh respectively, creating an iconic sex symbol for the new decade. Having run to embrace Dudley [SEP] tensor([0])
[CLS] How this movie escaped the wrath of MST3K I'll never know. " Gymkata " is a ridiculous action movie, filled ( or is that empty? ) with paper - thin plots, dumb characters, and preposterous situations. But take it from me, if you enjoy watching poor, yet goofy, movies, you will enjoy " Gymkata " a great deal. < br / > < br / > The action centers around a gymnast [SEP] tensor([0])
[CLS] I have to admit when I went to see this movie, I didn't really have high expectations. But even with my low expectations I was totally and utterly disa

In [9]:

data = make_batches(data_train_list, 16, 512)
bert = make_bert().cuda()
lossfn = nn.CrossEntropyLoss()

In [10]:

lr = 1e-5
optimizer = torch.optim.Adam(bert.parameters(), lr)

In [11]:
for param in bert.parameters():
    param.retain_grad()

In [12]:
import torchviz
viz = None
def train(model, optimizer, data, lossfn, epochs=1):
    global viz
    model.train()
    i = 0
    for epoch in range(epochs):
        for X, y in tqdm(data):

            optimizer.zero_grad()
            X = X.cuda()
            _lm_output, y_hat = model(X, only_classification=True)
            loss = lossfn(y_hat, y.cuda())
            loss.backward()
            optimizer.step()
            i += 1
            if i % 50 == 0:
                print(loss)
            #print(tokenizer.batch_decode(X[:2]), y_hat, y)
train(bert, optimizer, data, lossfn)

  0%|          | 0/1563 [00:00<?, ?it/s]

tensor(0.6023, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2978, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2693, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3743, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3547, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3671, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2577, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2695, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1723, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0852, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0700, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0795, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1168, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0732, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4169, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5320, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1620, device='cuda:0', grad_fn=

In [13]:
bert(torch.tensor(tokenizer.encode("I highly recommend this movie to my worst enemy.")).cuda().unsqueeze(0))[1].softmax(1)

tensor([[0.0837, 0.9163]], device='cuda:0', grad_fn=<SoftmaxBackward0>)